# TP2 Discord Chatbot - 13 November 2024 - Emma DESTÉ

<img src="screenshots/integrated_chatbot_framework.png" width="700" height="500">

## <span style="color:red; font-weight:bold;">1) Discord Bot </span>

### <span style="color:green; font-weight:bold;">1 - On Discord </span>

On the Discord developer portal https://discord.com/developers/applications, I logged in -> Applications -> New Application -> Create my app parrot_bot

At first, I really struggled to create an app. Because everytime I created one, it was always deleted when I refresh the page.


Solutions tested :
- Change my browser : neither Chrome or Firefox works -> no
- Incognito mode on my Safari browser -> no

On my Discord account :
- Advanced -> Activate developer mode
<img src="screenshots/1_parametre_du_compte.png" width="400" height="250">

- Confidentiality & Security -> User parameters -> Stop filtering the supposed spam messages 
<img src="screenshots/2_no_filtering.png" width="400" height="250">

- Confidentiality & Security -> verify my account is ok
<img src="screenshots/3_good_account.png" width="400" height="250">

- Server -> Copy server ID, I indeed can do it so this is not a developer mode problem linked to my account

****************

So desesperately, I create a new account and praying to be working this time. Update : it works !!!

Installation menu :
- **Install Link -> None**
- **Installation Contexts -> X User Install**

Bot menu :
- **disable : Public Bot**
- **enable : Message Content Intent**

Create a new Discord server Chatbot_EFREI, add my bot to the users of the server : Inviter des gens -> lien vers le serveur https://discord.gg/3SWJ9PCm

Back to the Developer Portal : **OAuth2 -> URL Generator -> Scopes : bot -> Bot permissions : administrator**
I paste the generated URL into a new window of my browser, and make my bot accept the invitation.

<img src="screenshots/addBot_1.png" width="200" height="150">
<img src="screenshots/addBot_2.png" width="200" height="150">

It works well, my bot is now part of my Discord server.

<img src="screenshots/addBot_3.png" width="200" height="150">


### <span style="color:green; font-weight:bold;">2 - On Python </span>

In [1]:
pip install -U discord

Note: you may need to restart the kernel to use updated packages.


Let's create the Discord connection :

My bot token is obtained on Discord Developer Portal : **Bot -> Token -> Reset Token**

/!\ It is important to note that it will not work in the notebook, so I create a Python file to achieve this on PyCharm.
However, I copy my code here so that it is easier to have a see of what I have done.

First, let's install discord library on my PyCharm Python environment : **Terminal -> pip install -U discord**

In [ ]:
# Step 1 : Implement an instance of Client

import discord

TOKEN = "MTMwNjI1NDYzNDg5MzE4MDkyOQ.GNZk3s.7jJSwTdDn5bj7LwhySSyMQ95MTZyRhzRurLquE"  # my bot token
class MyClient (discord . Client):
    async def on_ready(self):
        print('Logged in as ' + str(self.user.name))
        print('The chatbot Id :: ' + str(self.user.id))

client = MyClient(intents=discord.Intents.default())
client.run(TOKEN)

The output is : 
Logged in as parrot_bot

The chatbot Id : 1306254634893180929. So it works fine :)


### <span style="color:green; font-weight:bold;">3 - Practice</span>

First, I write on the general salon textuel so that the bot can respond to me :

In [ ]:
# Step 2 : Receiving and sending messages from my Discord server & chatbot doesn't interact with its own messages

# Inside class MyClient (discord.Client) I add :

async def on_message(self, message):

    if message.author == self.user:  # for the bot itself message -> ignore
        return
    
    if message.content.lower() == "hello":  # to understand the message even if it is capital letters
        await message.channel.send("Hello! How can I assist you today ?")

    if message.content.lower() == "how are you ?":
        await message.channel.send("Fine and you ? The weather is cool today ☀️")

<img src="screenshots/bot_response_1.png" width="300" height="80">

Second, I add my weather chatbot. I copy my code into a new Python file that contains all the packages and functions, of course I had to go into the environment Terminal and install the necessary packages : *nltk*, *spellchecker* and *indexer*, *random*, *requests*.

Of course I adapt my main code for having a Discord chat. My new function is then :

In [ ]:
# Inside class MyClient (discord.Client) 
# In async def on_message(self, message) I add :

async def on_message(self, message):

    if message.author == self.user:  # for the bot itself message -> ignore
        return

    if message.content.lower() == "how are you ?":  # to understand the message even if it is capital letters
        await message.channel.send("Fine and you ? The weather is cool today ☀️")

    else:
        # no need to do a loop anymore -> Discord already perform its own question-answer loop

        user_intention = new_intent_search(message.content)

        # no more quit option

        # 4) Call weather functions if intention detected is a question about weather
        if user_intention == 'weather question':
            city_time = createEntities(message.content)

            weather_info_json = getWeather(city_time['city'], city_time['time'])
            weather_response = detailed_temperature_response(weather_info_json)

            await message.channel.send(weather_response)

        # 2) Define a default response to the undefined intention from the user.
        elif user_intention == 'undefined':
            await message.channel.send("I did not understand. Could you tell me something else ?")
        else:
            # Répondre avec une réponse générique du chatbot
            response = chatbot_response(user_intention)
            await message.channel.send(response)

I implemented new functionalities : I tried to be as creative as I can :)
- 🖼️ **When the user asks for the weather, I display a picture of this city** (1)

I create an account so that I have an access key to this API. I also create an application on Unsplash in order for me to have an access key to call this API. 
Documentation here : https://unsplash.com/documentation, especially the search photo section

<img src="screenshots/city_image_2.png" width="300" height="80">

- 👤 **Saying the username when the user is greeting or leaving** (2)

<img src="screenshots/username_response.png" width="300" height="80">


- 🎞️ **Display a GIF when the user thanks, apologies or helps** (3)

I create an account on Tenor, create an app so that I have an access key to its API. 
The documentation to handle the use of the API is available here : https://developers.google.com/tenor/guides/quickstart?hl=fr

<img src="screenshots/gif_apology.png" width="300" height="80">


## <span style="color:red; font-weight:bold;">2) Wit.ai </span>

To extract intentions from the user, we defined regex in TP1, now we will use NLP to capture intent and entities much more easily and automatically.

### <span style="color:green; font-weight:bold;">1 - Change Chatbot topic </span>

Let's implement a Movie Bot.

First, we need the database API from https://www.themoviedb.org to get informations on movies : **Paramètres -> API -> Demander une clé d'API -> Développeur**. My access key is 731f1dd6e50b36840dc855649f445a3b

This documentation helps me to create my own queries : https://developer.themoviedb.org/docs/wrappers-and-libraries

In [2]:
pip install tmdbv3api

Note: you may need to restart the kernel to use updated packages.


In this section, I want to try to display the info about the movie and the poster from a defined user sentence.

In [120]:
from tmdbv3api import TMDb, Search, Movie

def query_tmdb_db(movie_name):
    # initialize the TMDb object with my API key
    tmdb = TMDb()
    tmdb.api_key = '731f1dd6e50b36840dc855649f445a3b'

    # create Movie object to interact with the Movie endpoints
    movie_api = Movie()

    # let's try manual try out
    response = Search().movies(movie_name)  # I get the warning : search() is depreciated, use Search().movies()

    for movie in response:
        movie_name = movie.title
        print(f"🎬 Title: {movie_name}")
        
        release_year = movie.release_date[:4]
        print(f"🗓️ Release Year: {release_year}")
        
        plot = movie.overview
        print(f"📖 Plot: {plot}")
        
        # rating = movie.vote_average
        
        if movie.poster_path:  # to avoid URL ending by None
            poster_link = f"https://image.tmdb.org/t/p/w500{movie.poster_path}"
            print(f"🖼️ Poster Path: {poster_link}")

        credits = movie_api.credits(movie.id)  # fetch movie credits using the movie ID
        director = None  # init
        if 'crew' in credits:
            for crew_member in credits['crew']:
                if crew_member['job'] == 'Director':
                    director = crew_member['name']
                    break  # exit when 1 director found
        print(f"🎥 Director: {director if director else 'No director found'}")

        print("-" * 20)
    

In [121]:
movie_name = "Star Wars"
query_tmdb_db(movie_name)

🎬 Title: Star Wars
🗓️ Release Year: 1977
📖 Plot: Princess Leia is captured and held hostage by the evil Imperial forces in their effort to take over the galactic Empire. Venturesome Luke Skywalker and dashing captain Han Solo team together with the loveable robot duo R2-D2 and C-3PO to rescue the beautiful princess and restore peace and justice in the Empire.
🖼️ Poster Path: https://image.tmdb.org/t/p/w500/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg
🎥 Director: George Lucas
--------------------
🎬 Title: Star Wars: The Force Awakens
🗓️ Release Year: 2015
📖 Plot: Thirty years after defeating the Galactic Empire, Han Solo and his allies face a new threat from the evil Kylo Ren and his army of Stormtroopers.
🖼️ Poster Path: https://image.tmdb.org/t/p/w500/wqnLdwVXoBjKibFRR5U3y0aDUhs.jpg
🎥 Director: J.J. Abrams
--------------------
🎬 Title: Star Wars: The Last Jedi
🗓️ Release Year: 2017
📖 Plot: Rey develops her newly discovered abilities with the guidance of Luke Skywalker, who is unsettled by the stre

In [83]:
# Displaying the first poster : Star Wars of 1977 (to check if the path generated is correct)

from IPython.display import Image, display

url = "https://image.tmdb.org/t/p/w500/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg"
display(Image(url=url))

### <span style="color:green; font-weight:bold;">2 - Starting a Wit.ai App </span>

Let's use the LNU : Wit.ai

### <span style="color:green; font-weight:bold;">3 - Train it to detect Intents and Entities </span>

First, I sign in to my Meta account on https://wit.ai and **New App -> Understanding**

Then I add one by one the sentence with their according intent and entities.
<img src="screenshots/wit_new_sentence.png" width="600" height="200">

Then it displays the following message to ensure that the sentences are well registered :
<img src="screenshots/wit_new_added.png" width="500" height="200">

Since it is based on ML and AI, the more I give examples the more the model gets accurate. So I define more sentences that are similar to the ones in the TP.

********

I define sentences for the intents like director, realease year, and also more basic intents like greeting, bye, help and thanks.
With some changes of syntax so that my database contains better data. 

Here are the counts of the sentences I defined one by one : 34 for movie info, 34 for the director, 38 for the actors, 22 for the release year, 22 for greeting, 26 for bye, 31 for help and 32 for thanks.
Totally, I now get **239 sentences and intents recorded in my database**. Hope it will be good enough to create a quite performant robot.

Let see the whole list of intents and entities that I have given to the model : **Management -> Intents**
<img src="screenshots/wit_list_intents2.png" width="180" height="50">

As well as the sentences : **Management -> Utterances**
<img src="screenshots/wit_list_utt2.png" width="170" height="50">

********

Also, I observed that as long as I was recording more and more sentences, the confidence level was also increasing along and it finished by giving me most of the time the correct intent. It is nice to see that the practice match the theory.

### <span style="color:green; font-weight:bold;">4 - Query Wit from Python </span>

First of all, beginning this part, I realize that my output looks like this when I call the API : so that entities are always empty

<img src="screenshots/wit_badEnti.png" width="170" height="50">

Meaning that I have misunderstood the guidelines and then wrongly registered sentences in my database. It should be train and validate like the following :

<img src="screenshots/wit_goodDef.png" width="170" height="50">

So now... I just have to delete my previous trainings and do it again with a correct registration method. I love being me :)

In [6]:
pip install wit

Note: you may need to restart the kernel to use updated packages.


In [51]:
from wit import Wit

wit_access_token = "XGC7EWF2AKPW4FVQ7PHASOZE7TO6ZS72"  # I define my access key to wit.ai

client = Wit(wit_access_token)
nlp_data = client.message ('tell me about the Spiderman (2007)')
print(nlp_data)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wit.ai:443
DEBUG:urllib3.connectionpool:https://api.wit.ai:443 "GET /message?q=tell+me+about+the+Spiderman+%282007%29 HTTP/1.1" 200 863


{'entities': {'movieName:movieName': [{'body': 'the Spiderman', 'confidence': 0.9995, 'end': 27, 'entities': [], 'id': '1613627029224984', 'name': 'movieName', 'role': 'movieName', 'start': 14, 'type': 'value', 'value': 'the spiderman'}], 'releaseYear:releaseYear': [{'body': '(2007)', 'confidence': 1, 'end': 34, 'entities': [], 'id': '1205896443820124', 'name': 'releaseYear', 'role': 'releaseYear', 'start': 28, 'type': 'value', 'value': '(2007)'}]}, 'intents': [{'confidence': 0.9981838130946519, 'id': '1915679975610770', 'name': 'movieinfo'}], 'text': 'tell me about the Spiderman (2007)', 'traits': {}}


So the wit API is well accessed since I get a complete response. I am happy to obtain all the info that I need for later : confidence level and intent of the message. Let's now create the function to extract the entities, because I just cannot give this type of raw answer to a user. Let's proccess :

In [37]:
def extract_entity(nlp_data, entity_name):

    entities = nlp_data.get('entities', {})
    entity_key = f"{entity_name}:{entity_name}"  # format 'entities': {'movieName:movieName'
    
    if entity_key in entities:
        for entity in entities[entity_key]:  # iterate through entity entries
            if entity.get('confidence', 0) >= 0.5:  # verify the confi level is enough
                return entity.get('value')
    
    return None

Now let's test this function to verify I well obtained the movies' titles for all the intents that I defined in the database.

In [47]:
from wit import Wit

wit_access_token = "XGC7EWF2AKPW4FVQ7PHASOZE7TO6ZS72"  # I define my access key to wit.ai

client = Wit(wit_access_token)
nlp_data = client.message('tell me about the Spiderman (2007)')
print(nlp_data)

entity_value = extract_entity(nlp_data, "movieName")
print(entity_value)

entity_value = extract_entity(nlp_data, "releaseYear")
print(entity_value)  # Output: 2007

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wit.ai:443
DEBUG:urllib3.connectionpool:https://api.wit.ai:443 "GET /message?q=tell+me+about+the+Spiderman+%282007%29 HTTP/1.1" 200 868


{'entities': {'movieName:movieName': [{'body': 'the Spiderman', 'confidence': 0.9995, 'end': 27, 'entities': [], 'id': '1613627029224984', 'name': 'movieName', 'role': 'movieName', 'start': 14, 'type': 'value', 'value': 'the spiderman'}], 'releaseYear:releaseYear': [{'body': '(2007)', 'confidence': 0.9995, 'end': 34, 'entities': [], 'id': '1205896443820124', 'name': 'releaseYear', 'role': 'releaseYear', 'start': 28, 'type': 'value', 'value': '(2007)'}]}, 'intents': [{'confidence': 0.9991085139271106, 'id': '1915679975610770', 'name': 'movieinfo'}], 'text': 'tell me about the Spiderman (2007)', 'traits': {}}
the spiderman
(2007)


In [48]:
nlp_data = client.message ('Can you tell me the story of Frozen?')
print(nlp_data)

entity_value = extract_entity(nlp_data, "movieName")
print(entity_value)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wit.ai:443
DEBUG:urllib3.connectionpool:https://api.wit.ai:443 "GET /message?q=Can+you+tell+me+the+story+of+Frozen%3F HTTP/1.1" 200 806


{'entities': {'movieName:movieName': [{'body': 'story', 'confidence': 1, 'end': 25, 'entities': [], 'id': '1613627029224984', 'name': 'movieName', 'role': 'movieName', 'start': 20, 'type': 'value', 'value': 'Story'}, {'body': 'Frozen', 'confidence': 0.9995, 'end': 35, 'entities': [], 'id': '1613627029224984', 'name': 'movieName', 'role': 'movieName', 'start': 29, 'type': 'value', 'value': 'Frozen'}]}, 'intents': [{'confidence': 0.9921682584616702, 'id': '1915679975610770', 'name': 'movieinfo'}], 'text': 'Can you tell me the story of Frozen?', 'traits': {}}
Story


In [49]:
nlp_data = client.message ('Who’s the mastermind behind Gladiator?')
print(nlp_data)

entity_value = extract_entity(nlp_data, "movieName")
print(entity_value)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wit.ai:443
DEBUG:urllib3.connectionpool:https://api.wit.ai:443 "GET /message?q=Who%E2%80%99s+the+mastermind+behind+Gladiator%3F HTTP/1.1" 200 542


{'entities': {'movieName:movieName': [{'body': 'Gladiator', 'confidence': 0.9995, 'end': 37, 'entities': [], 'id': '1613627029224984', 'name': 'movieName', 'role': 'movieName', 'start': 28, 'type': 'value', 'value': 'Gladiator'}]}, 'intents': [{'confidence': 0.9801335217566862, 'id': '1150079026534659', 'name': 'director'}], 'text': 'Who’s the mastermind behind Gladiator?', 'traits': {}}
Gladiator


In [50]:
nlp_data = client.message ('any idea when Titanic came out?')
print(nlp_data)

entity_value = extract_entity(nlp_data, "movieName")
print(entity_value)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wit.ai:443
DEBUG:urllib3.connectionpool:https://api.wit.ai:443 "GET /message?q=any+idea+when+Titanic+came+out%3F HTTP/1.1" 200 531


{'entities': {'movieName:movieName': [{'body': 'Titanic', 'confidence': 0.9995, 'end': 21, 'entities': [], 'id': '1613627029224984', 'name': 'movieName', 'role': 'movieName', 'start': 14, 'type': 'value', 'value': 'Titanic'}]}, 'intents': [{'confidence': 0.8700780745493157, 'id': '549794091248342', 'name': 'releaseYear'}], 'text': 'any idea when Titanic came out?', 'traits': {}}
Titanic


Now, I want to try to display the info about the movie and the poster accordingly to a defined user sentence and its intent. So let's redefined a function that doesn't display all the info of the movie :

In [212]:
def movie_entity_from_intent(movie_name, intent):
    # initialize the TMDb object with my API key
    tmdb = TMDb()
    tmdb.api_key = '731f1dd6e50b36840dc855649f445a3b'

    # create Movie object to interact with the Movie endpoints
    movie_api = Movie()

    # let's try manual try out
    response = Search().movies(movie_name)  # I get the warning : search() is depreciated, use Search().movies()

    for movie in response:
        
        movie_name = movie.title
        
        release_year = movie.release_date[:4]
        
        plot = movie.overview
        
        if movie.poster_path:  # to avoid URL ending by None
            poster_link = f"https://image.tmdb.org/t/p/w500{movie.poster_path}"

        credits = movie_api.credits(movie.id)  # fetch movie credits using the movie ID
        director = None  # init
        if 'crew' in credits:
            for crew_member in credits['crew']:
                if crew_member['job'] == 'Director':
                    director = crew_member['name']
                    break  # exit when 1 director found
                    
        main_cast = ""
        if 'cast' in credits:
            #print(credits)
            actor_names = ", ".join(actor['name'] for actor in credits['cast'])

            comma_position = [i for i, c in enumerate(actor_names) if c == ',']
            if len(comma_position) >= 5:  # keep only 5 for lisibility
                main_cast = actor_names[:comma_position[4]]

                    
        if intent == "movieinfo":  # return plot & poster / releaseYear possible
            return plot, poster_link

        elif intent == "director":  # return director & poster
            return director, poster_link
        
        elif intent == "actors":  # return actors & poster
            return main_cast, poster_link

        elif intent == "releaseYear":  # return releaseYear & poster
            return release_year, poster_link

        else:  # if none of the intents match
            return None


In [141]:
entity, poster = movie_entity_from_intent("Frozen", "movieinfo")
print(entity, poster)

Young princess Anna of Arendelle dreams about finding true love at her sister Elsa’s coronation. Fate takes her on a dangerous journey in an attempt to end the eternal winter that has fallen over the kingdom. She's accompanied by ice delivery man Kristoff, his reindeer Sven, and snowman Olaf. On an adventure where she will find out what friendship, courage, family, and true love really means. https://image.tmdb.org/t/p/w500/mbPrrbt8bSLcHSBCHnRclPlMZPl.jpg


In [143]:
entity, poster = movie_entity_from_intent("Star Wars", "director")
print(entity, poster)

George Lucas https://image.tmdb.org/t/p/w500/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg


In [147]:
entity, poster = movie_entity_from_intent("Frozen", "releaseYear")
print(entity, poster)

2013 https://image.tmdb.org/t/p/w500/mbPrrbt8bSLcHSBCHnRclPlMZPl.jpg


In [214]:
entity, poster = movie_entity_from_intent("Titanic", "actors")
print(entity, poster)

Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates, Frances Fisher https://image.tmdb.org/t/p/w500/9xjZS2rlVxm8SFx8kPC3aIGCOYQ.jpg


## <span style="color:red; font-weight:bold;">3) Bringing it all together </span>

I now have : 
- a channel on Discord
- a trained NLP system on Wit.ai
- an API as a database

First, let's define the different functions I will need in order to make my main working.

In [221]:
# 3) Capture intent and entities (if any)

def extract_intent(nlp_data):
    return nlp_data['intents'][0]['name']


nlp_data = client.message('What is Frozen about?')
intent_name = extract_intent(nlp_data)
print(intent_name)

nlp_data = client.message('Who directed Frozen?')
intent_name = extract_intent(nlp_data)
print(intent_name)

nlp_data = client.message('Who are the main actors in La La Land?')
intent_name = extract_intent(nlp_data)
print(intent_name)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wit.ai:443
DEBUG:urllib3.connectionpool:https://api.wit.ai:443 "GET /message?q=What+is+Frozen+about%3F HTTP/1.1" 200 517
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wit.ai:443


movieinfo


DEBUG:urllib3.connectionpool:https://api.wit.ai:443 "GET /message?q=Who+directed+Frozen%3F HTTP/1.1" 200 516
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wit.ai:443


director


DEBUG:urllib3.connectionpool:https://api.wit.ai:443 "GET /message?q=Who+are+the+main+actors+in+La+La+Land%3F HTTP/1.1" 200 538


actors


In [215]:
# 5) Rephrase the data into a full-sentence response

def rephrase_response(intent, movie, entity, poster):
                
    if intent == "movieinfo":
        answer = f"🎬 The plot of the movie {movie} is : {entity}. To have a better look on the story, here is the poster: {poster}"

    elif intent == "director":
        answer = f"🎥 This is {entity} who directed the fantastical movie {movie}. Have a look at his vision by seeing the movie's poster : {poster}"

    elif intent == "actors":    
        answer = f"👤 The movie {movie} is wonderful also thanks to the actors who played in it : {entity}. You can clearly recognize them on the movie's poster: {poster}"
        
    elif intent == "releaseYear":
        answer = f"🗓️ The movie {movie} was released in {entity}. The atmosphere of that era is reflected on its poster: {poster}"

    else:
        return None

    return answer

In [175]:
def main_movie(discord_input):
    # 2) Send it to the Wit.ai model
    wit_access_token = "XGC7EWF2AKPW4FVQ7PHASOZE7TO6ZS72"  # I define my access key to wit.ai
    client = Wit(wit_access_token)
    nlp_data = client.message(discord_input)
    # print(nlp_data)

    
    # 3) Capture intent and entities (if any)
    # 4) Query TheMovieDB API to retrieve the required data
    intent_found = extract_intent(nlp_data)
    print(intent_found)

    movieName = extract_entity(nlp_data, "movieName")  # f° from section 2.3
    print(movieName)

    #releaseYear = extract_entity(nlp_data, "releaseYear")
    #print(releaseYear)

    entity, poster = movie_entity_from_intent(movieName, intent_found)
    # print(entity, poster)

    
    # 5) Rephrase the data into a full-sentence response
    reply = rephrase_response(intent_found, movieName, entity, poster)

    # 6) Reply to the user on Discord
    print(reply)


In [176]:
# 1) Receive the user’s message input on Discord
# For the notebook I define a sentence
discord_input = "Tell me about The Spiderman (2007)"

main_movie(discord_input)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wit.ai:443
DEBUG:urllib3.connectionpool:https://api.wit.ai:443 "GET /message?q=Tell+me+about+The+Spiderman+%282007%29 HTTP/1.1" 200 858


movieinfo
the spiderman
🎬 The plot of the movie the spiderman is : After reuniting with Gwen Stacy, Brooklyn’s full-time, friendly neighborhood Spider-Man is catapulted across the Multiverse, where he encounters the Spider Society, a team of Spider-People charged with protecting the Multiverse’s very existence. But when the heroes clash on how to handle a new threat, Miles finds himself pitted against the other Spiders and must set out on his own to save those he loves most.. To have a better look on the story, here is the poster: https://image.tmdb.org/t/p/w500/8Vt6mWEReuy4Of61Lnj5Xj704m8.jpg


In [177]:
discord_input2 = "Who directed Gladiator?"

main_movie(discord_input2)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wit.ai:443
DEBUG:urllib3.connectionpool:https://api.wit.ai:443 "GET /message?q=Who+directed+Gladiator%3F HTTP/1.1" 200 525
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.themoviedb.org:443


director
Gladiator


DEBUG:urllib3.connectionpool:https://api.themoviedb.org:443 "GET /3/search/movie?api_key=731f1dd6e50b36840dc855649f445a3b&query=Gladiator&page=1&language=en-US HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.themoviedb.org:443
DEBUG:urllib3.connectionpool:https://api.themoviedb.org:443 "GET /3/movie/98/credits?api_key=731f1dd6e50b36840dc855649f445a3b&&language=en-US HTTP/1.1" 200 None


🎥 This is Ridley Scott who directed the fantastical movie Gladiator. Have a look at his vision by seeing the movie's poster : https://image.tmdb.org/t/p/w500/ty8TGRuvJLPUmAR1H1nRIsgwvim.jpg


In [178]:
discord_input3 = "when Titanic released in theaters?"

main_movie(discord_input3)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wit.ai:443
DEBUG:urllib3.connectionpool:https://api.wit.ai:443 "GET /message?q=when+Titanic+released+in+theaters%3F HTTP/1.1" 200 533
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.themoviedb.org:443
DEBUG:urllib3.connectionpool:https://api.themoviedb.org:443 "GET /3/search/movie?api_key=731f1dd6e50b36840dc855649f445a3b&query=Titanic&page=1&language=en-US HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.themoviedb.org:443


releaseYear
Titanic


DEBUG:urllib3.connectionpool:https://api.themoviedb.org:443 "GET /3/movie/597/credits?api_key=731f1dd6e50b36840dc855649f445a3b&&language=en-US HTTP/1.1" 200 None


🗓️ The movie Titanic was released in 1997. The atmosphere of that era is reflected on its poster: https://image.tmdb.org/t/p/w500/9xjZS2rlVxm8SFx8kPC3aIGCOYQ.jpg


In [220]:
discord_input4 = "Who are the main actors in La La Land?"

main_movie(discord_input4)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wit.ai:443
DEBUG:urllib3.connectionpool:https://api.wit.ai:443 "GET /message?q=Who+are+the+main+actors+in+La+La+Land%3F HTTP/1.1" 200 538
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.themoviedb.org:443
DEBUG:urllib3.connectionpool:https://api.themoviedb.org:443 "GET /3/search/movie?api_key=731f1dd6e50b36840dc855649f445a3b&query=La%20La%20Land&page=1&language=en-US HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.themoviedb.org:443


{'entities': {'movieName:movieName': [{'body': 'La La Land', 'confidence': 0.9995, 'end': 37, 'entities': [], 'id': '1613627029224984', 'name': 'movieName', 'role': 'movieName', 'start': 27, 'type': 'value', 'value': 'La La Land'}]}, 'intents': [{'confidence': 0.967946648766318, 'id': '485052284640176', 'name': 'actors'}], 'text': 'Who are the main actors in La La Land?', 'traits': {}}
actors
La La Land


DEBUG:urllib3.connectionpool:https://api.themoviedb.org:443 "GET /3/movie/313369/credits?api_key=731f1dd6e50b36840dc855649f445a3b&&language=en-US HTTP/1.1" 200 None


👤 The movie La La Land is wonderful also thanks to the actors who played in it : Ryan Gosling, Emma Stone, John Legend, Rosemarie DeWitt, J.K. Simmons. You can clearly recognize them on the movie's poster: https://image.tmdb.org/t/p/w500/uDO8zWDhfWwoFdKS4fzkUJt0Rf0.jpg
